In [6]:
from pathlib import Path
from scipy.io import loadmat
import sys
import os


dataset_path = Path('data') / 'data.mat'
if not dataset_path.exists():
    alt = Path.cwd().parent / 'data' / 'data.mat'
    if alt.exists():
        dataset_path = alt
    else:
        raise FileNotFoundError(f"data.mat not found under {Path.cwd()} or its parent")

notebook_path = os.getcwd() 
print (f"Current notebook path: {notebook_path}")
project_root = os.path.dirname(notebook_path)
if project_root not in sys.path:
    sys.path.insert(0, project_root)
print (f"Added {project_root} to sys.path")

mat_data = loadmat(dataset_path)
print(mat_data.keys())

Current notebook path: /home/luky/skola/KalmanNet-for-state-estimation/TAN
Added /home/luky/skola/KalmanNet-for-state-estimation to sys.path
dict_keys(['__header__', '__version__', '__globals__', 'hB', 'souradniceGNSS', 'souradniceX', 'souradniceY', 'souradniceZ'])


In [7]:
import torch
import matplotlib.pyplot as plt
from utils import trainer
from utils import utils
from Systems import DynamicSystem
import Filters
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from scipy.io import loadmat
from scipy.interpolate import RegularGridInterpolator
import random

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Používané zařízení: {device}")

Používané zařízení: cuda


In [8]:
mat_data = loadmat(dataset_path)

souradniceX_mapa = mat_data['souradniceX']
souradniceY_mapa = mat_data['souradniceY']
souradniceZ_mapa = mat_data['souradniceZ']
souradniceGNSS = mat_data['souradniceGNSS'] 
x_axis_unique = souradniceX_mapa[0, :]
y_axis_unique = souradniceY_mapa[:, 0]

print(f"Rozměry 1D osy X: {x_axis_unique.shape}")
print(f"Rozměry 1D osy Y: {y_axis_unique.shape}")
print(f"Rozměry 2D dat výšek Z: {souradniceZ_mapa.shape}")


terMap_interpolator = RegularGridInterpolator(
    (y_axis_unique, x_axis_unique),
    souradniceZ_mapa,
    bounds_error=False, 
    fill_value=np.nan
)
print("...interpolační funkce vytvořena.")

def terMap(px, py):

    points_to_query = np.column_stack((py, px))
    
    return terMap_interpolator(points_to_query)

Rozměry 1D osy X: (2500,)
Rozměry 1D osy Y: (2500,)
Rozměry 2D dat výšek Z: (2500, 2500)
...interpolační funkce vytvořena.


# 4D model


In [9]:
import torch
from math import pi
from Systems import DynamicSystemTAN

state_dim = 4
obs_dim = 3
dT = 1
q = 1

F = torch.tensor([[1.0, 0.0, dT, 0.0],
                   [0.0, 1.0, 0.0, dT],
                   [0.0, 0.0, 1.0, 0.0],
                   [0.0, 0.0, 0.0, 1.0]])

Q = q* torch.tensor([[dT**3/3, 0.0, dT**2/2, 0.0],
                   [0.0, dT**3/3, 0.0, dT**2/2],
                   [dT**2/2, 0.0, dT, 0.0],
                   [0.0, dT**2/2, 0.0, dT]])
R = torch.tensor([[3.0**2, 0.0, 0.0],
                   [0.0, 1.0**2, 0.0],
                   [0.0, 0.0, 1.0**2]])

initial_velocity_np = souradniceGNSS[:2, 1] - souradniceGNSS[:2, 0]
initial_velocity = torch.from_numpy(initial_velocity_np)

initial_position = torch.from_numpy(souradniceGNSS[:2, 0])
x_0 = torch.cat([
    initial_position,
    initial_velocity
]).float()
print(x_0)

P_0 = torch.tensor([[25.0, 0.0, 0.0, 0.0],
                    [0.0, 25.0, 0.0, 0.0],
                    [0.0, 0.0, 0.5, 0.0],
                    [0.0, 0.0, 0.0, 0.5]])

def h_nl_robust(x: torch.Tensor) -> torch.Tensor:
    # ... (implementace s clampingem, jak jsme si ukázali dříve) ...
    # Získání hranic mapy
    min_x, max_x = x_axis_unique.min(), x_axis_unique.max()
    min_y, max_y = y_axis_unique.min(), y_axis_unique.max()

    # Oříznutí pozic POUZE pro dotaz do mapy
    px_safe = x[:, 0].clone().clamp(min_x, max_x)
    py_safe = x[:, 1].clone().clamp(min_y, max_y)
    vel_safe = 200.0 # ciste kvuli numericke explozi

    vyska_terenu_np = terMap(px_safe.detach().cpu().numpy(), py_safe.detach().cpu().numpy())
    vyska_terenu = torch.from_numpy(vyska_terenu_np).float().to(x.device)
    
    # Zbytek výpočtu s původními rychlostmi
    eps = 1e-12


    vx_w, vy_w = x[:, 2], x[:, 3]
    vx_w = x[:, 2].clamp(-vel_safe, vel_safe)
    vy_w = x[:, 3].clamp(-vel_safe, vel_safe)
    norm_v_w = torch.sqrt(vx_w**2 + vy_w**2).clamp(min=eps)
    cos_psi = vx_w / norm_v_w
    sin_psi = vy_w / norm_v_w
    vx_b = cos_psi * vx_w - sin_psi * vy_w
    vy_b = sin_psi * vx_w + cos_psi * vy_w
    
    result = torch.stack([vyska_terenu, vx_b, vy_b], dim=1)
    
    # Pojistka pro případ, že by terMap přesto vrátila NaN
    if torch.isnan(result).any():
        print("Varování: NaN hodnoty v měření detekovány, nahrazuji nulami.")
        result[torch.isnan(result)] = 0

    return result
import torch.nn.functional as func

def h_nl_differentiable(x: torch.Tensor, map_tensor, x_min, x_max, y_min, y_max) -> torch.Tensor:
    batch_size = x.shape[0]

    px = x[:, 0]
    py = x[:, 1]

    px_norm = 2.0 * (px - x_min) / (x_max - x_min) - 1.0
    py_norm = 2.0 * (py - y_min) / (y_max - y_min) - 1.0

    sampling_grid = torch.stack((px_norm, py_norm), dim=1).view(batch_size, 1, 1, 2)

    vyska_terenu_batch = func.grid_sample(
        map_tensor.expand(batch_size, -1, -1, -1),
        sampling_grid, 
        mode='bilinear', 
        padding_mode='border',
        align_corners=True
    )

    vyska_terenu = vyska_terenu_batch.view(batch_size)

    eps = 1e-12
    vx_w, vy_w = x[:, 2], x[:, 3]
    vel_safe = 100.0 # ciste kvuli numericke explozi
    vx_w = x[:, 2].clamp(-vel_safe, vel_safe)
    vy_w = x[:, 3].clamp(-vel_safe, vel_safe)
    norm_v_w = torch.sqrt(vx_w**2 + vy_w**2).clamp(min=eps)
    cos_psi = vx_w / norm_v_w
    sin_psi = vy_w / norm_v_w

    vx_b = cos_psi * vx_w - sin_psi * vy_w 
    vy_b = sin_psi * vx_w + cos_psi * vy_w

    result = torch.stack([vyska_terenu, vx_b, vy_b], dim=1)

    return result

x_axis_unique = souradniceX_mapa[0, :]
y_axis_unique = souradniceY_mapa[:, 0]
terMap_tensor = torch.from_numpy(souradniceZ_mapa).float().unsqueeze(0).unsqueeze(0).to(device)
x_min, x_max = x_axis_unique.min(), x_axis_unique.max()
y_min, y_max = y_axis_unique.min(), y_axis_unique.max()

h_wrapper = lambda x: h_nl_differentiable(
    x, 
    map_tensor=terMap_tensor, 
    x_min=x_min, 
    x_max=x_max, 
    y_min=y_min, 
    y_max=y_max
)

system_model = DynamicSystemTAN(
    state_dim=state_dim,
    obs_dim=obs_dim,
    Q=Q.float(),
    R=R.float(),
    Ex0=x_0.float(),
    P0=P_0.float(),
    F=F.float(),
    h=h_wrapper,
    x_axis_unique=x_axis_unique, 
    y_axis_unique=y_axis_unique,
    device=device
)

tensor([ 1.4875e+06,  6.3955e+06,  4.3225e+00, -4.1456e+01])
INFO: DynamicSystemTAN inicializován s hranicemi mapy:
  X: [1476611.42, 1489541.47]
  Y: [6384032.63, 6400441.34]


In [10]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from Systems import DynamicSystemTAN 
from copy import deepcopy 
import numpy as np
import random 
from utils import utils 
TRAIN_SEQ_LEN = 200 
VALID_SEQ_LEN = 200
NUM_TRAIN_SETS = 60
TRAJ_PER_SET_TRAIN = 7
NUM_VALID_SETS = 20
TRAJ_PER_SET_VALID = 7
BATCH_SIZE = 256 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Používané zařízení: {device}")

num_gnss_points = souradniceGNSS.shape[1]
print(f"Načteno {num_gnss_points} GNSS bodů pro výběr počátečních podmínek.")
original_system_model = system_model 
default_P0 = torch.diag(torch.tensor([25.0, 25.0, 0.5, 0.5], device=device)).float()

print("Generuji trénovací data s náhodnými počátečními podmínkami z GNSS...")
all_x_train = []
all_y_train = []
for i in range(NUM_TRAIN_SETS):
    print(f"  Generuji trénovací sadu {i+1}/{NUM_TRAIN_SETS}...")


    start_index = random.randint(0, num_gnss_points - 2)

    initial_pos_np = souradniceGNSS[:2, start_index]
    next_pos_np = souradniceGNSS[:2, start_index + 1]
    initial_vel_np = next_pos_np - initial_pos_np
    Ex0_sampled = torch.cat([
        torch.from_numpy(initial_pos_np),
        torch.from_numpy(initial_vel_np)
    ]).float().to(device)

    P0_current = default_P0

    temp_model = deepcopy(original_system_model)
    temp_model.Ex0 = Ex0_sampled
    temp_model.P0 = P0_current

    x_batch, y_batch = utils.generate_data_for_map(
        temp_model,
        num_trajectories=TRAJ_PER_SET_TRAIN,
        seq_len=TRAIN_SEQ_LEN
    )
    all_x_train.append(x_batch)
    all_y_train.append(y_batch)

x_train = torch.cat(all_x_train, dim=0)
y_train = torch.cat(all_y_train, dim=0)
print(f"Finální trénovací data: x={x_train.shape}, y={y_train.shape}")

print("Generuji validační data s náhodnými počátečními podmínkami z GNSS...")
all_x_val = []
all_y_val = []
for i in range(NUM_VALID_SETS):
    print(f"  Generuji validační sadu {i+1}/{NUM_VALID_SETS}...")
    start_index = random.randint(0, num_gnss_points - 2)
    initial_pos_np = souradniceGNSS[:2, start_index]
    next_pos_np = souradniceGNSS[:2, start_index + 1]
    initial_vel_np = next_pos_np - initial_pos_np 
    Ex0_sampled = torch.cat([
        torch.from_numpy(initial_pos_np),
        torch.from_numpy(initial_vel_np)
    ]).float().to(device)
    P0_current = default_P0 

    temp_model = deepcopy(original_system_model)
    temp_model.Ex0 = Ex0_sampled
    temp_model.P0 = P0_current

    x_batch, y_batch = utils.generate_data_for_map(
        temp_model,
        num_trajectories=TRAJ_PER_SET_VALID,
        seq_len=VALID_SEQ_LEN
    )
    all_x_val.append(x_batch)
    all_y_val.append(y_batch)

x_val = torch.cat(all_x_val, dim=0)
y_val = torch.cat(all_y_val, dim=0)
print(f"Finální validační data: x={x_val.shape}, y={y_val.shape}")

print("\nPočítám normalizační statistiky (průměr a std) z trénovacích dat...")

# Získáme state_dim (což je 4) z modelu
state_dim = original_system_model.state_dim

# Tvar x_train je [N_traj, Seq_Len, State_Dim]
# Tvar x_train_flat bude [N_traj * Seq_Len, State_Dim]
x_train_flat = x_train.view(-1, state_dim)

# Vypočítáme průměr a std pro každou ze 4 komponent stavu
x_mean = x_train_flat.mean(dim=0).to(device)
x_std = x_train_flat.std(dim=0).to(device)

x_std[x_std == 0] = 1.0

print(f"  Vypočtený průměr (x_mean): {x_mean.cpu().numpy()}")
print(f"  Vypočtená odchylka (x_std): {x_std.cpu().numpy()}")

train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

print("\nDataLoadery jsou připraveny pro trénink.")

TEST_SEQ_LEN = 600 
NUM_TEST_TRAJ = 10

print(f"\nGeneruji {NUM_TEST_TRAJ} testovacích trajektorií o délce {TEST_SEQ_LEN}...")

x_test, y_test = utils.generate_data_for_map(
    system_model, 
    num_trajectories=NUM_TEST_TRAJ,
    seq_len=TEST_SEQ_LEN
)

test_dataset = TensorDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

Používané zařízení: cuda
Načteno 1276 GNSS bodů pro výběr počátečních podmínek.
Generuji trénovací data s náhodnými počátečními podmínkami z GNSS...
  Generuji trénovací sadu 1/60...
INFO: Generátor dat používá hranice X:[1476611.42-1489541.47], Y:[6384032.63-6400441.34]
Generuji 7 platných trajektorií (metoda zahození)...
  Úspěšně vygenerována trajektorie 1/7 (Pokusů: 3)
  Úspěšně vygenerována trajektorie 2/7 (Pokusů: 5)
  Úspěšně vygenerována trajektorie 3/7 (Pokusů: 6)
  Úspěšně vygenerována trajektorie 4/7 (Pokusů: 9)
  Úspěšně vygenerována trajektorie 5/7 (Pokusů: 12)
  Úspěšně vygenerována trajektorie 6/7 (Pokusů: 17)
  Úspěšně vygenerována trajektorie 7/7 (Pokusů: 18)
------------------------------
Generování dat dokončeno.
Celkový počet pokusů: 18
Úspěšnost (platné trajektorie / pokusy): 38.89%
Celkový počet vygenerovaných trajektorií: torch.Size([7, 200, 4])
  Generuji trénovací sadu 2/60...
INFO: Generátor dat používá hranice X:[1476611.42-1489541.47], Y:[6384032.63-6400441.

In [11]:
# import torch
# import torch.nn as nn
# from torch.utils.data import TensorDataset, DataLoader
# import numpy as np
# import os
# import random
# from copy import deepcopy
# from state_NN_models import StateKalmanNet_v2 
# from utils import trainer 

# torch.manual_seed(42)
# np.random.seed(42)
# random.seed(42)

# state_knet2 = StateKalmanNet_v2(
#     system_model=original_system_model, 
#     device=device,
#     hidden_size_multiplier=12,
#     output_layer_multiplier=4,
#     num_gru_layers=2
# ).to(device)

# trained_model = trainer.train_state_KalmanNet_sliding_window(
#     model=state_knet2,
#     train_loader=train_loader,
#     val_loader=val_loader,
#     device=device,
#     epochs=200,
#     lr=1e-3,
#     clip_grad=1.0,
#     early_stopping_patience=20,
#     tbptt_k=2,
#     tbptt_w=8,
#     optimizer_=torch.optim.AdamW,
#     weight_decay_=1e-3,

# )

In [12]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from IPython.display import display
import time
from state_NN_models import StateKalmanNet_v2
from utils import trainer

# --- 1. Definice prostoru pro HPO (Grid Search) ---
print("Spouštím Grid Search pro HPO...")
start_time_hpo = time.time()

# Zde definujte hodnoty, které chcete testovat
hidden_multipliers = [2,4, 6, 8, 10] 
output_multipliers = [1, 2, 4]
gru_hidden_dim_multipliers = [4]

results_list = []

# --- 2. Spuštění smyček Grid Search ---
total_runs = len(hidden_multipliers) * len(output_multipliers) * len(gru_hidden_dim_multipliers)
run_count = 0

for h_mult in hidden_multipliers:
    for o_mult in output_multipliers:
        for g_mult in gru_hidden_dim_multipliers: 
            run_count += 1
            run_id = f"h{h_mult}_o{o_mult}_g{g_mult}"
            print(f"\n{'='*80}")
            print(f"Běh HPO {run_count}/{total_runs}: {run_id} (Hidden: {h_mult}, Output: {o_mult}, GRU: {g_mult})")
            print(f"{'='*80}")
            
            start_time_run = time.time()
            
            # Inicializace metrik pro případ selhání
            best_train_loss = float('inf')
            best_val_loss = float('inf')
            final_test_mse = float('inf')

            # --- 3. BLOK TRY...EXCEPT PRO ODCHYCENÍ NESTABILITY ---
            try:
                # --- 3a. Inicializace modelu ---
                current_model = StateKalmanNet_v2(
                    system_model=original_system_model, 
                    device=device,
                    hidden_size_multiplier=h_mult,
                    output_layer_multiplier=o_mult,
                    gru_hidden_dim_multiplier=g_mult, 
                    num_gru_layers=1
                ).to(device)
                print(f"Model inicializován: {current_model}")

                # --- 3b. Trénování modelu ---
                training_results = trainer.train_state_KalmanNet_sliding_window_grid_search(
                    model=current_model,
                    train_loader=train_loader,
                    val_loader=val_loader,
                    device=device,
                    epochs=200, 
                    lr=1e-4,
                    clip_grad=1.0,
                    early_stopping_patience=30, 
                    tbptt_k=2,
                    tbptt_w=8,
                    optimizer_=torch.optim.AdamW,
                    weight_decay_=1e-4,
                    verbose=False
                )
                
                best_train_loss = training_results['best_train_loss']
                best_val_loss = training_results['best_val_loss']
                current_model = training_results['model']
                
                print(f"Trénování dokončeno. Nejlepší Train Loss: {best_train_loss:.6f}, Val Loss: {best_val_loss:.6f}")

                # --- 3c. Evaluace na testovací sadě ---
                print("Evaluace na testovacích datech...")
                current_model.eval() 
                
                test_mse_list = []
                with torch.no_grad():
                    for x_true_seq_batch, y_test_seq_batch in test_loader:
                        y_test_seq_gpu = y_test_seq_batch.squeeze(0).to(device)
                        x_true_seq_gpu = x_true_seq_batch.squeeze(0).to(device)
                        initial_state = x_true_seq_gpu[0, :].unsqueeze(0)
                        TEST_SEQ_LEN = x_true_seq_gpu.shape[0] 

                        current_model.reset(batch_size=1, initial_state=initial_state)
                        model_preds = []
                        for t in range(1, TEST_SEQ_LEN):
                            step_output = current_model.step(y_test_seq_gpu[t, :].unsqueeze(0))
                            
                            if current_model.returns_covariance:
                                x_filtered_t = step_output[0]
                            else:
                                x_filtered_t = step_output
                            model_preds.append(x_filtered_t)
                        
                        full_x_hat_model = torch.cat([initial_state, torch.cat(model_preds, dim=0)], dim=0)
                        mse = F.mse_loss(full_x_hat_model[1:], x_true_seq_gpu[1:]).item()
                        
                        # Zkontrolujeme, zda i MSE není náhodou NaN/Inf
                        if not np.isfinite(mse):
                            print("Varování: MSE na testovací sadě je NaN/Inf!")
                            raise RuntimeError("Selhání při evaluaci (NaN MSE)")
                            
                        test_mse_list.append(mse)

                final_test_mse = np.mean(test_mse_list)
                print(f"Evaluace dokončena. Průměrné Test MSE: {final_test_mse:.6f}.")

            # --- 3d. Zpracování výjimky (když se objeví NaN) ---
            except RuntimeError as e:
                if "NaN" in str(e) or "Inf" in str(e):
                    print(f"\n!!!!!!!!!!!!!!!!! POZOR !!!!!!!!!!!!!!!!!")
                    print(f"Běh {run_id} selhal kvůli numerické nestabilitě (NaN/Inf).")
                    print(f"Chyba: {e}")
                    print(f"Tento běh bude zaznamenán s MSE = 'inf' a HPO bude pokračovat.")
                    print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\n")
                    final_test_mse = float('inf') # Penalizace
                else:
                    # Pokud je to jiná chyba, chceme, aby HPO spadlo
                    raise e 
            
            run_duration = time.time() - start_time_run
            print(f"Doba trvání běhu: {run_duration:.2f}s")
            
            # --- 6. Uložení výsledků ---
            results_list.append({
                "run_id": run_id,
                "h_mult": h_mult,
                "o_mult": o_mult,
                "g_mult": g_mult,
                "best_train_loss": best_train_loss,
                "best_val_loss": best_val_loss,
                "test_mse": final_test_mse,
                "duration_s": run_duration
            })

# --- 7. Zobrazení finální tabulky ---
print("\n" + "="*80)
print(f"Grid Search HPO Dokončen! Celkový čas: {(time.time() - start_time_hpo) / 60:.2f} minut.")
print("="*80)

# Vytvoření a seřazení DataFrame
results_df = pd.DataFrame(results_list)
results_df = results_df.sort_values(by="test_mse", ascending=True)

pd.set_option('display.float_format', '{:.6f}'.format)
display(results_df)

print("\nNejlepší konfigurace (podle Test MSE):")
print(results_df.iloc[0])

Spouštím Grid Search pro HPO...

Běh HPO 1/15: h2_o1_g4 (Hidden: 2, Output: 1, GRU: 4)
Model inicializován: StateKalmanNet_v2(
  (dnn): DNN_KalmanNet_v2(
    (input_norm): LayerNorm((14,), eps=1e-05, elementwise_affine=True)
    (input_layer): Sequential(
      (0): Linear(in_features=14, out_features=112, bias=True)
      (1): ReLU()
    )
    (gru): GRU(112, 100)
    (output_hidden_layer): Sequential(
      (0): Linear(in_features=100, out_features=12, bias=True)
      (1): ReLU()
    )
    (output_final_linear): Linear(in_features=12, out_features=12, bias=True)
  )
)


/home/luky/.local/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Trénování dokončeno. Nejlepší Train Loss: 2085793.681426, Val Loss: 1715528.375000
Evaluace na testovacích datech...
Evaluace dokončena. Průměrné Test MSE: 10879598.062500.
Doba trvání běhu: 183.41s

Běh HPO 2/15: h2_o2_g4 (Hidden: 2, Output: 2, GRU: 4)
Model inicializován: StateKalmanNet_v2(
  (dnn): DNN_KalmanNet_v2(
    (input_norm): LayerNorm((14,), eps=1e-05, elementwise_affine=True)
    (input_layer): Sequential(
      (0): Linear(in_features=14, out_features=112, bias=True)
      (1): ReLU()
    )
    (gru): GRU(112, 100)
    (output_hidden_layer): Sequential(
      (0): Linear(in_features=100, out_features=24, bias=True)
      (1): ReLU()
    )
    (output_final_linear): Linear(in_features=24, out_features=12, bias=True)
  )
)
Trénování dokončeno. Nejlepší Train Loss: 128408878871.445343, Val Loss: 291743662080.000000
Evaluace na testovacích datech...
Evaluace dokončena. Průměrné Test MSE: 5735227772108.799805.
Doba trvání běhu: 48.33s

Běh HPO 3/15: h2_o4_g4 (Hidden: 2, Output

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from IPython.display import display
import time
from state_NN_models import StateKalmanNet_arch2
from utils import trainer

# --- 1. Definice prostoru pro HPO (Grid Search) ---
print("Spouštím Grid Search pro HPO...")
start_time_hpo = time.time()

# Zde definujte hodnoty, které chcete testovat
hidden_multipliers = [4,6,8] 
output_multipliers = [2,4,6]
results_list = []

# --- 2. Spuštění smyček Grid Search ---
total_runs = len(hidden_multipliers) * len(output_multipliers)
run_count = 0

for h_mult in hidden_multipliers:
    for o_mult in output_multipliers:
            run_count += 1
            run_id = f"h{h_mult}_o{o_mult}"
            print(f"\n{'='*80}")
            print(f"Běh HPO {run_count}/{total_runs}: {run_id} (Hidden: {h_mult}, Output: {o_mult})")
            print(f"{'='*80}")
            
            start_time_run = time.time()
            
            # Inicializace metrik pro případ selhání
            best_train_loss = float('inf')
            best_val_loss = float('inf')
            final_test_mse = float('inf')

            # --- 3. BLOK TRY...EXCEPT PRO ODCHYCENÍ NESTABILITY ---
            try:
                # --- 3a. Inicializace modelu ---
                current_model = StateKalmanNet_arch2(
                    system_model=original_system_model, 
                    device=device,
                    hidden_size_multiplier=h_mult,
                    output_layer_multiplier=o_mult
                ).to(device)
                print(f"Model inicializován: {current_model}")

                # --- 3b. Trénování modelu ---
                training_results = trainer.train_state_KalmanNet_sliding_window_grid_search(
                    model=current_model,
                    train_loader=train_loader,
                    val_loader=val_loader,
                    device=device,
                    epochs=200, 
                    lr=1e-4,
                    clip_grad=1.0,
                    early_stopping_patience=30, 
                    tbptt_k=2,
                    tbptt_w=8,
                    optimizer_=torch.optim.AdamW,
                    weight_decay_=1e-4,
                    verbose=False
                )
                
                best_train_loss = training_results['best_train_loss']
                best_val_loss = training_results['best_val_loss']
                current_model = training_results['model']
                
                print(f"Trénování dokončeno. Nejlepší Train Loss: {best_train_loss:.6f}, Val Loss: {best_val_loss:.6f}")

                # --- 3c. Evaluace na testovací sadě ---
                print("Evaluace na testovacích datech...")
                current_model.eval() 
                
                test_mse_list = []
                with torch.no_grad():
                    for x_true_seq_batch, y_test_seq_batch in test_loader:
                        y_test_seq_gpu = y_test_seq_batch.squeeze(0).to(device)
                        x_true_seq_gpu = x_true_seq_batch.squeeze(0).to(device)
                        initial_state = x_true_seq_gpu[0, :].unsqueeze(0)
                        TEST_SEQ_LEN = x_true_seq_gpu.shape[0] 

                        current_model.reset(batch_size=1, initial_state=initial_state)
                        model_preds = []
                        for t in range(1, TEST_SEQ_LEN):
                            step_output = current_model.step(y_test_seq_gpu[t, :].unsqueeze(0))
                            
                            if current_model.returns_covariance:
                                x_filtered_t = step_output[0]
                            else:
                                x_filtered_t = step_output
                            model_preds.append(x_filtered_t)
                        
                        full_x_hat_model = torch.cat([initial_state, torch.cat(model_preds, dim=0)], dim=0)
                        mse = F.mse_loss(full_x_hat_model[1:], x_true_seq_gpu[1:]).item()
                        
                        # Zkontrolujeme, zda i MSE není náhodou NaN/Inf
                        if not np.isfinite(mse):
                            print("Varování: MSE na testovací sadě je NaN/Inf!")
                            raise RuntimeError("Selhání při evaluaci (NaN MSE)")
                            
                        test_mse_list.append(mse)

                final_test_mse = np.mean(test_mse_list)
                print(f"Evaluace dokončena. Průměrné Test MSE: {final_test_mse:.6f}.")

            # --- 3d. Zpracování výjimky (když se objeví NaN) ---
            except RuntimeError as e:
                if "NaN" in str(e) or "Inf" in str(e):
                    print(f"\n!!!!!!!!!!!!!!!!! POZOR !!!!!!!!!!!!!!!!!")
                    print(f"Běh {run_id} selhal kvůli numerické nestabilitě (NaN/Inf).")
                    print(f"Chyba: {e}")
                    print(f"Tento běh bude zaznamenán s MSE = 'inf' a HPO bude pokračovat.")
                    print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\n")
                    final_test_mse = float('inf') # Penalizace
                else:
                    # Pokud je to jiná chyba, chceme, aby HPO spadlo
                    raise e 
            
            run_duration = time.time() - start_time_run
            print(f"Doba trvání běhu: {run_duration:.2f}s")
            
            # --- 6. Uložení výsledků ---
            results_list.append({
                "run_id": run_id,
                "h_mult": h_mult,
                "o_mult": o_mult,
                "best_train_loss": best_train_loss,
                "best_val_loss": best_val_loss,
                "test_mse": final_test_mse,
                "duration_s": run_duration
            })

# --- 7. Zobrazení finální tabulky ---
print("\n" + "="*80)
print(f"Grid Search HPO Dokončen! Celkový čas: {(time.time() - start_time_hpo) / 60:.2f} minut.")
print("="*80)

# Vytvoření a seřazení DataFrame
results_df = pd.DataFrame(results_list)
results_df = results_df.sort_values(by="test_mse", ascending=True)

pd.set_option('display.float_format', '{:.6f}'.format)
display(results_df)

print("\nNejlepší konfigurace (podle Test MSE):")
print(results_df.iloc[0])